In [8]:
!pip install gymnasium

In [17]:
graphs = [
    {
        "name": "G1",
        "adj_matrix": [
            [0, 1, 0, 0],
            [1, 0, 1, 0],
            [0, 1, 0, 1],
            [0, 0, 1, 0]
        ],
        "start": 0,
        "end": 3
    },
    {
        "name": "G2",
        "adj_matrix": [
            # 0 1 2 3 4 5 6 7 8
            [0, 1, 1, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 1, 1, 0, 0, 0, 0],
            [1, 0, 0, 0, 1, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 1, 1, 0],
            [0, 1, 1, 0, 0, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 1],
            [0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 0, 0, 0]
        ],
        "start": 0,
        "end": 8
    }
]

In [18]:
results = []

In [19]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class GraphEnv(gym.Env):

    def __init__(self, adj_matrix, start_node, end_node):
        super(GraphEnv, self).__init__()

        self.adj_matrix = np.array(adj_matrix)
        self.num_nodes = len(adj_matrix)
        self.start_node = start_node
        self.end_node = end_node

        self.current_node = self.start_node

        self.action_space = spaces.Discrete(self.num_nodes)
        self.observation_space = spaces.Discrete(self.num_nodes)

        # Step counter to prevent infinite loops
        self.max_steps = self.num_nodes * 2
        self.step_count = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_node = self.start_node
        self.step_count = 0
        return self.current_node, {}

    def step(self, action):

        self.step_count += 1
        terminated = False
        truncated = False

        # Check if neighbor
        if self.adj_matrix[self.current_node][action] == 1:
            self.current_node = action
            reward = -1
        else:
            reward = -10

        if self.current_node == self.end_node:
            terminated = True
            reward = 0

        if self.step_count >= self.max_steps:
            truncated = True

        return self.current_node, reward, terminated, truncated, {}

# **Q-Learning**

In [20]:
import time

for g in graphs:
    adj_matrix = g["adj_matrix"]
    START_NODE = g["start"]
    END_NODE = g["end"]

    env = GraphEnv(adj_matrix, START_NODE, END_NODE)
    q_table = np.zeros([env.num_nodes, env.num_nodes])

    # RL hyperparameters
    learning_rate = 0.1
    discount = 0.95
    epochs = 1000
    epsilon = 1.0
    epsilon_decay = 0.99

    start_time = time.time()

    for epoch in range(epochs):
        state = env.reset()[0]
        terminated = False
        truncated = False

        while not terminated and not truncated:
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state])

            new_state, reward, terminated, truncated, _ = env.step(action)
            old_q = q_table[state, action]
            max_future_q = np.max(q_table[new_state])
            q_table[state, action] = old_q + learning_rate * (reward + discount * max_future_q - old_q)
            state = new_state

        epsilon = max(0.01, epsilon * epsilon_decay)

    elapsed_time = time.time() - start_time

    # Extract path
    path = [START_NODE]
    state = START_NODE
    while state != END_NODE:
        action = int(np.argmax(q_table[state]))
        path.append(action)
        state = action
        if len(path) > env.num_nodes:
            print(f"Warning: RL stuck in loop on {g['name']}")
            break

    # Store RL results
    results.append({
        "graph": g["name"],
        "method": "RL",
        "path": path,
        "path_length": len(path)-1,
        "training_time": elapsed_time
    })

In [15]:
# env = GraphEnv(adj_matrix, START_NODE, END_NODE)

# q_table = np.zeros([env.num_nodes, env.num_nodes])

# learning_rate = 0.1
# discount = 0.95
# epochs = 1000
# epsilon = 1.0
# epsilon_decay = 0.99

# for epoch in range(epochs):
#     state = env.reset()[0]
#     terminated = False
#     truncated = False

#     while not terminated and not truncated:
#         # Epsilon-greedy
#         if np.random.random() < epsilon:
#             action = env.action_space.sample()
#         else:
#             action = np.argmax(q_table[state])

#         new_state, reward, terminated, truncated, _ = env.step(action)

#         old_q = q_table[state, action]
#         max_future_q = np.max(q_table[new_state])

#         new_q = old_q + learning_rate * (reward + discount * max_future_q - old_q)
#         q_table[state, action] = new_q

#         state = new_state

#     epsilon = max(0.01, epsilon * epsilon_decay)

# print("RL Training finished!")
# print("Final Q-Table:")
# print(q_table.round(2))

# path = [START_NODE]
# state = START_NODE
# while state != END_NODE:
#     action = int(np.argmax(q_table[state]))
#     path.append(action)
#     state = action
#     if len(path) > env.num_nodes:
#         print("Error: Pathfinding stuck in a loop.")
#         break
# print(f"Shortest path found by RL: {path}")

RL Training finished!
Final Q-Table:
[[-11.19  -2.85  -2.85 -11.31 -10.86 -11.08 -10.49 -11.74 -11.97]
 [ -2.4  -10.95 -10.39  -2.38  -1.95 -10.07 -11.06 -10.43 -11.11]
 [ -2.85  -7.87  -8.73  -8.88  -1.95  -8.48  -9.53  -6.96  -9.47]
 [ -8.24  -2.31  -4.1   -7.83  -6.02  -7.54  -2.29  -2.33  -7.25]
 [ -9.03  -2.28  -2.49  -8.34  -9.55  -1.    -9.19  -9.56  -8.38]
 [ -7.94  -8.15  -9.11  -7.94  -1.6   -7.18  -8.5   -7.71   0.  ]
 [ -5.63  -3.63  -2.89  -2.3   -6.25  -3.66  -2.89  -2.92  -3.57]
 [ -6.5   -3.6   -2.99  -2.35  -4.2   -4.21  -5.32  -4.71  -6.27]
 [  0.     0.     0.     0.     0.     0.     0.     0.     0.  ]]
Shortest path found by RL: [0, 1, 4, 5, 8]


# **DRL**

In [21]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

# --- Q-Network ---
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# --- Replay Buffer ---
class ReplayBuffer:
    def __init__(self, buffer_size, batch_size):
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size

    def add(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def sample(self):
        experiences = random.sample(self.memory, self.batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences])).float()
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences])).long()
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences])).float()
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences])).float()
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences]).astype(np.uint8)).float()
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)

# --- Helper function: one-hot encoding of node ---
def one_hot(node, num_nodes):
    vec = np.zeros(num_nodes, dtype=float)
    vec[node] = 1.0
    return vec

# --- DRL parameters ---
BUFFER_SIZE = 10000
BATCH_SIZE = 32
GAMMA = 0.95
LR = 0.001
EPSILON = 1.0
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.995
EPISODES = 500
UPDATE_EVERY = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- DRL for all graphs ---
for g in graphs:
    adj_matrix = g["adj_matrix"]
    START_NODE = g["start"]
    END_NODE = g["end"]

    env = GraphEnv(adj_matrix, START_NODE, END_NODE)
    state_size = env.num_nodes
    action_size = env.num_nodes

    # Q-networks and optimizer
    q_local = QNetwork(state_size, action_size).to(device)
    q_target = QNetwork(state_size, action_size).to(device)
    optimizer = optim.Adam(q_local.parameters(), lr=LR)
    memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE)
    q_target.load_state_dict(q_local.state_dict())

    epsilon = EPSILON
    start_time = time.time()

    # --- Training loop ---
    for episode in range(1, EPISODES + 1):
        state = env.reset()[0]
        state_vec = one_hot(state, state_size)
        terminated = False
        truncated = False

        while not terminated and not truncated:
            state_tensor = torch.from_numpy(state_vec).float().unsqueeze(0).to(device)
            q_local.eval()
            with torch.no_grad():
                action_values = q_local(state_tensor)
            q_local.train()

            if random.random() > epsilon:
                action = np.argmax(action_values.cpu().data.numpy())
            else:
                action = random.choice(np.arange(action_size))

            next_state, reward, terminated, truncated, _ = env.step(action)
            next_state_vec = one_hot(next_state, state_size)
            done = terminated or truncated

            memory.add(state_vec, action, reward, next_state_vec, done)

            if len(memory) > BATCH_SIZE:
                experiences = memory.sample()

                # --- Learn function ---
                states, actions, rewards, next_states, dones = experiences
                states = states.to(device)
                actions = actions.to(device)
                rewards = rewards.to(device)
                next_states = next_states.to(device)
                dones = dones.to(device)

                best_actions = q_local(next_states).detach().argmax(1).unsqueeze(1)
                q_targets_next = q_target(next_states).detach().gather(1, best_actions)
                q_targets = rewards + GAMMA * q_targets_next * (1 - dones)
                q_expected = q_local(states).gather(1, actions)

                loss = nn.MSELoss()(q_expected, q_targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Soft update
                for target_param, local_param in zip(q_target.parameters(), q_local.parameters()):
                    target_param.data.copy_(0.01 * local_param.data + (1.0 - 0.01) * target_param.data)

            state_vec = next_state_vec

        epsilon = max(EPSILON_MIN, epsilon * EPSILON_DECAY)

    elapsed_time = time.time() - start_time

    # --- Extract shortest path ---
    path = [START_NODE]
    state_vec = one_hot(START_NODE, state_size)
    state = START_NODE
    while state != END_NODE:
        state_tensor = torch.from_numpy(state_vec).float().unsqueeze(0).to(device)
        q_local.eval()
        with torch.no_grad():
            action = torch.argmax(q_local(state_tensor)).item()
        path.append(action)
        state_vec = one_hot(action, state_size)
        state = action
        if len(path) > env.num_nodes:
            print(f"Warning: DRL stuck in loop on {g['name']}")
            break

    # --- Store DRL results ---
    results.append({
        "graph": g["name"],
        "method": "DRL",
        "path": path,
        "path_length": len(path)-1,
        "training_time": elapsed_time
    })

Using device: cuda


In [24]:
from tabulate import tabulate

table_data = []
for r in results:
    table_data.append([
        r['graph'],
        r['method'],
        str(r['path']),
        r['path_length'],
        f"{r['training_time']:.2f}s"
    ])

headers = ["Graph", "Method", "Path", "Path Length", "Training Time"]

print(tabulate(table_data, headers=headers, tablefmt="grid"))

+---------+----------+-----------------+---------------+-----------------+
| Graph   | Method   | Path            |   Path Length | Training Time   |
+=========+==========+=================+===============+=================+
| G1      | RL       | [0, 1, 2, 3]    |             3 | 0.03s           |
+---------+----------+-----------------+---------------+-----------------+
| G2      | RL       | [0, 1, 4, 5, 8] |             4 | 0.05s           |
+---------+----------+-----------------+---------------+-----------------+
| G1      | DRL      | [0, 1, 2, 3]    |             3 | 7.59s           |
+---------+----------+-----------------+---------------+-----------------+
| G2      | DRL      | [0, 2, 4, 5, 8] |             4 | 13.01s          |
+---------+----------+-----------------+---------------+-----------------+


In [16]:
# import gymnasium as gym
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# import random
# from collections import deque

# # Q-Network
# class QNetwork(nn.Module):
#     def __init__(self, state_size, action_size):
#         super(QNetwork, self).__init__()
#         self.fc1 = nn.Linear(state_size, 64)
#         self.fc2 = nn.Linear(64, 64)
#         self.fc3 = nn.Linear(64, action_size)

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         return self.fc3(x)

# # Replay Buffer
# class ReplayBuffer:
#     def __init__(self, buffer_size, batch_size):
#         self.memory = deque(maxlen=buffer_size)
#         self.batch_size = batch_size

#     def add(self, state, action, reward, next_state, done):
#         self.memory.append((state, action, reward, next_state, done))

#     def sample(self):
#         experiences = random.sample(self.memory, self.batch_size)
#         states = torch.from_numpy(np.vstack([e[0] for e in experiences])).float()
#         actions = torch.from_numpy(np.vstack([e[1] for e in experiences])).long()
#         rewards = torch.from_numpy(np.vstack([e[2] for e in experiences])).float()
#         next_states = torch.from_numpy(np.vstack([e[3] for e in experiences])).float()
#         dones = torch.from_numpy(np.vstack([e[4] for e in experiences]).astype(np.uint8)).float()
#         return (states, actions, rewards, next_states, dones)

#     def __len__(self):
#         return len(self.memory)

# env = GraphEnv(adj_matrix, START_NODE, END_NODE)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# state_size = env.num_nodes
# action_size = env.num_nodes

# q_local = QNetwork(state_size, action_size).to(device)
# q_target = QNetwork(state_size, action_size).to(device)
# optimizer = optim.Adam(q_local.parameters(), lr=0.001)
# memory = ReplayBuffer(10000, 32)
# q_target.load_state_dict(q_local.state_dict())

# GAMMA = 0.95
# EPSILON = 1.0
# EPSILON_MIN = 0.01
# EPSILON_DECAY = 0.995
# UPDATE_EVERY = 4
# episodes = 500

# def one_hot(node, num_nodes):
#     vec = np.zeros(num_nodes, dtype=float)
#     vec[node] = 1.0
#     return vec

# def learn(experiences, gamma):
#     states, actions, rewards, next_states, dones = experiences
#     states = states.to(device)
#     actions = actions.to(device)
#     rewards = rewards.to(device)
#     next_states = next_states.to(device)
#     dones = dones.to(device)

#     best_actions = q_local(next_states).detach().argmax(1).unsqueeze(1)
#     q_targets_next = q_target(next_states).detach().gather(1, best_actions)
#     q_targets = rewards + gamma * q_targets_next * (1 - dones)
#     q_expected = q_local(states).gather(1, actions)

#     loss = nn.MSELoss()(q_expected, q_targets)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     # Soft update
#     for target_param, local_param in zip(q_target.parameters(), q_local.parameters()):
#         target_param.data.copy_(0.01 * local_param.data + (1.0 - 0.01) * target_param.data)

# # Training loop
# for episode in range(1, episodes + 1):
#     state = env.reset()[0]
#     state_vec = one_hot(state, state_size)
#     terminated = False
#     truncated = False

#     while not terminated and not truncated:
#         state_tensor = torch.from_numpy(state_vec).float().unsqueeze(0).to(device)
#         q_local.eval()
#         with torch.no_grad():
#             action_values = q_local(state_tensor)
#         q_local.train()

#         if random.random() > EPSILON:
#             action = np.argmax(action_values.cpu().data.numpy())
#         else:
#             action = random.choice(np.arange(action_size))

#         next_state, reward, terminated, truncated, _ = env.step(action)
#         next_state_vec = one_hot(next_state, state_size)
#         done = terminated or truncated

#         memory.add(state_vec, action, reward, next_state_vec, done)

#         if len(memory) > 32:
#             experiences = memory.sample()
#             learn(experiences, GAMMA)

#         state_vec = next_state_vec

#     EPSILON = max(EPSILON_MIN, EPSILON * EPSILON_DECAY)

#     if episode % 50 == 0:
#         print(f"Episode {episode}/{episodes}")

# path = [START_NODE]
# state_vec = one_hot(START_NODE, state_size)
# state = START_NODE
# while state != END_NODE:
#     state_tensor = torch.from_numpy(state_vec).float().unsqueeze(0).to(device)
#     q_local.eval()
#     with torch.no_grad():
#         action = torch.argmax(q_local(state_tensor)).item()
#     path.append(action)
#     state_vec = one_hot(action, state_size)
#     state = action
#     if len(path) > env.num_nodes:
#         print("Stuck in loop!")
#         break

# print(f"Shortest path found by DRL: {path}")

Using device: cuda
Episode 50/500
Episode 100/500
Episode 150/500
Episode 200/500
Episode 250/500
Episode 300/500
Episode 350/500
Episode 400/500
Episode 450/500
Episode 500/500
Shortest path found by DRL: [0, 2, 4, 5, 8]
